In [1]:
!pip3 install PyPDF2
!pip3 install fitz
!pip3 install PyMuPDF
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install pytesseract
!apt install ghostscript
!pip install ocrmypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 44.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (13.8 MB/s)
Selecting p

In [9]:
from google.colab import auth, drive
auth.authenticate_user()

from google.auth import default
creds, _ = default()

import PyPDF2
import pandas as pd
import fitz
import os
from ctypes.util import find_library
find_library("gs")

%load_ext google.colab.data_table

drive.mount('/content/drive')

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##OCR Sample Files

In [ ]:
file_list = []

input_folder = "/content/drive/My Drive/Doc Samples"
output_folder = "/content/drive/My Drive/Output"

for filename in os.listdir(input_folder):
    if filename.endswith("pdf"):
      file_list.append(filename)

print("The following files will be converted : ")
pd.DataFrame(file_list, columns=["Filename"])

In [ ]:
for filename in file_list:
  print("Converting : " + filename)
  input_filename = input_folder+'/'+filename
  output_filename = output_folder+'/'+filename
  !ocrmypdf --force-ocr "{input_filename}" "{output_filename}"

#Get Information
##Extract Text

In [ ]:
def extract_text(pdf):
  for i in range(pdf.page_count):
      data = []
      data.append(file.replace(".pdf", ""))
      data.append(i+1)
      page = pdf.load_page(i)
      txt = page.get_text()
      data.append(txt)
      df_text.loc[len(df_text.index)] = data

output_folder = "/content/drive/My Drive/OCR Samples"
pages = {'File':[],
              'Page':[],
            'Text': []}
df_text = pd.DataFrame(pages)

for file in os.listdir(output_folder):
  output_file = output_folder+'/'+file
  pdf =  fitz.open(output_file)
  extract_text(pdf)

df_text.to_csv('Delete.csv', index=False, escapechar='\\')

#NO LONGER NEEDED!!!

## Grab Bookmarks

In [12]:
def show_tree(bookmark_list):
    for item in bookmark_list:
            data = []
            data.append(item.title)
            data.append(item.page)
            df_book.loc[len(df_book.index)] = data

In [13]:
bookmark = {
                'Bookmark': [],
                'Page':[]}
df_book = pd.DataFrame(bookmark)
reader = PyPDF2.PdfReader(output_file)
show_tree(reader.outline)

In [15]:
df_book.to_csv('/content/drive/My Drive/Bookmarks.csv', index=False)

In [ ]:
result = pd.merge(df_text, df_book, on="Page")

##Merge all Sample Docs

In [4]:
merger = PyPDF2.PdfMerger()
output_folder = "/content/drive/My Drive/OCR Samples"
for file in os.listdir(output_folder):
  merger.append('/content/drive/MyDrive/OCR Samples/{}'.format(file), outline_item=file.replace(".pdf", ""), import_outline=False, pages=(0,1))

In [5]:
output_file = '/content/drive/My Drive/Sample_OCR.pdf'

merger.write(output_file)

merger.close()


##Shrink PDF Size

### Delete Comments and Bookmarks from File

In [ ]:
def remove_data(input_pdf, output_pdf):
    with open(input_pdf, 'rb') as infile:
        reader = PyPDF2.PdfReader(infile)
        writer = PyPDF2.PdfWriter()
        writer.addPage(reader.pages[0])
        for page_num in range(1, len(reader.pages)):
            page = reader.pages[page_num]
            if page.annotations:
                page.annotations.clear()
            writer.add_page(page)
        with open(output_pdf, 'wb') as outfile:
            writer.write(outfile)
    print("Comments and Bookmarks removed successfully!")

In [ ]:
remove_data( '/content/drive/My Drive/Samples.pdf', '/content/drive/My Drive/Clean_Sample.pdf')

### Compress File

In [ ]:
def compress_pdf(input_pdf, output_pdf):
    with open(input_pdf, 'rb') as infile:
        reader = PyPDF2.PdfReader(infile)
        writer = PyPDF2.PdfWriter()
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page.compress_content_streams()
            writer.add_page(page)
        with open(output_pdf, 'wb') as outfile:
            writer.write(outfile)
    print("PDF compressed successfully!")
    if output_pdf.stat().st_size > 524288000:
        compress_pdf(output_pdf, output_pdf)

In [ ]:
compress_pdf( '/content/drive/My Drive/Clean_Sample.pdf', '/content/drive/My Drive/Compressed_Sample.pdf')

##Combine Dfs